0 导入包

In [1]:
# 加载模型
from transformers import pipeline

# 加载数据
from transformers import AutoFeatureExtractor
from datasets import load_dataset
from datasets import Audio


# 随机测试数据
import random

# 播放数据
from IPython.display import Audio as AU # 接收(array，int)，输出音频

# 计算资源
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

In [2]:
# 对比模型地址
ft_model_id = "fted-gtzan/checkpoint-560"
ori_model_id = "ntu-spml/distilhubert"

1 加载数据集

In [3]:
gtzan = load_dataset("marsyas/gtzan", "all") # 1000个30s歌曲片段，分10个类别的数据集
gtzan = gtzan.cast_column("audio", Audio(sampling_rate=16000)) # 调整采样率
gtzan = gtzan.rename_column("genre", "label") # 重命名
gtzan

d:\env_config\py\lib\site-packages\datasets\load.py:1491: FutureWarning: The repository for marsyas/gtzan contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/marsyas/gtzan
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label'],
        num_rows: 999
    })
})

2 随机挑选数据进行比较

In [4]:
# 随机挑选数据进行测试
i = random.randint(1,999)
array = gtzan['train'][i]['audio']['array']
# 播放音乐
print(f'{i}答案：',
      gtzan["train"].features["label"].names[gtzan['train'][i]['label']])
AU(array, rate=16000) # 播放

566答案： jazz


In [5]:
# 原模型
aaa = pipeline("audio-classification",model =ori_model_id,device=device)
aaa(array) # 输入音频的numpy数组，输出预测结果

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\env_config\py\lib\site-packages\transformers\models\hubert\modeling_hubert.py:857: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[{'score': 0.5152433514595032, 'label': 'LABEL_0'},
 {'score': 0.4847566485404968, 'label': 'LABEL_1'}]

In [6]:
# 训练后模型
bbb = pipeline("audio-classification",model =ft_model_id ,device=device)
bbb(array) # 输入音频的numpy数组，输出预测结果

[{'score': 0.689245343208313, 'label': 'jazz'},
 {'score': 0.10972990840673447, 'label': 'blues'},
 {'score': 0.043890178203582764, 'label': 'country'},
 {'score': 0.03031954914331436, 'label': 'pop'},
 {'score': 0.03005347028374672, 'label': 'classical'}]

3 做成一个demo

In [7]:
import gradio as gr

# 加载模型
model_id = "fted-gtzan/checkpoint-560"
pipe = pipeline("audio-classification", model=model_id)

# 定义分类函数
def classify(filepath):
    a = pipe(filepath)
    outputs = {}
    for p in a:
        outputs[p["label"]] = p["score"]
    return outputs

demo = gr.Interface(
    fn=classify, 
    inputs=gr.Audio(type="filepath"), 
    outputs=gr.Label()
)

demo.launch(debug=True)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


d:\env_config\py\lib\site-packages\gradio\processing_utils.py:588: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
